# 2 Get commits

In [ ]:
get_commits()

4073 docker-java/docker-java 1790


In [1]:
import re
import time
import datetime
from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output

import sqlite3
con = sqlite3.connect('git.db')
write = con.cursor()
get = con.cursor()

from github import Github
user = 'dkbrz'
password = 'mandarinka24'
g = Github(user, password)


In [23]:
(g.get_rate_limit().core.reset).minute

34

In [2]:
class Repo:
    
    def __init__(self, R, count):
        self.name = R.name
        self.owner = R.owner.login
        self.full = R.full_name
        self.language = R.language
        self.lang_url = R.languages_url
        self.network = R.network_count
        self.stars = R.stargazers_count
        self.subs = R.subscribers_count
        self.watch = R.watchers_count
        self.created = R.created_at
        self.updated = R.updated_at
        self.size = R.size
        self.has_wiki = int(R.has_wiki)
        self.commit = count
        
    def __str__(self):
        return 'INSERT into R (name, owner, full, language, lang_url, network, stars, subs, \
        watch, created, updated, size, wiki, n_commits) \
        VALUES (\"{}\",\"{}\",\"{}\",\"{}\",\"{}\",{},{},{},{},\"{}\",\"{}\",{},{},{})'.format(
            str(self.name),
            str(self.owner),
            str(self.full),
            str(self.language),
            str(self.lang_url),
            str(self.network),
            str(self.stars),
            str(self.subs),
            str(self.watch),
            str(self.created),
            str(self.updated),
            str(self.size),
            str(self.has_wiki),
            str(self.commit)
            )

class Commit:
    
    def __init__(self, C, name):
        try:
            self.user = C.author.login
        except:
            try:
                self.user = C.committer.login
            except:
                self.user = ''
        self.date = C.commit.author.date
        self.message = C.commit.message.strip().replace('"','""').replace('\'','\'\'').replace('\\','\\\\')
        self.message = re.sub('-{2,}','\-\-', self.message)
        self.repo = name
        
    def __str__(self):
        return 'INSERT into C (user, full, date, message) \
        VALUES (\"{}\",\"{}\",\"{}\",\"{}\")'.format(
            str(self.user),
            str(self.repo),
            str(self.date),
            str(self.message)
            )

def get_commits():
    global get, write
    k = 0
    for line in open('full.txt','r'):
        name = line.strip()
        clear_output()
        try:
            Rate = g.get_rate_limit().core
            if Rate.remaining < 5:
                time.sleep((Rate.reset - datetime.datetime.utcnow()).seconds+1)
            R = g.get_repo(name)
            C = R.get_commits()
            count = C.totalCount
            if R.fork:
                print ('fork!')
                break
            else:
                print (k, line.strip(), count)
                if count > 480000: 
                    print ('Too much!')
                    with open ('log.txt', 'a') as f:
                        f.write('{}\t{}\n'.format(name, str(count)))
                else:
                    Rate = g.get_rate_limit().core
                    if count // 100 + 3 >= Rate.remaining:
                        print ('Well, lets wait!', str(Rate.reset))
                        time.sleep((Rate.reset - datetime.datetime.utcnow()).seconds+1)
                        C = R.get_commits()
                    write.execute('BEGIN TRANSACTION;')
                    C._PaginatedList__requester.per_page = 100
                    if count > 50000:
                        with open('./queries/{}.txt'.format(name.replace('/','-')), 'w') as statement:
                            statement.write('BEGIN TRANSACTION;\n')
                            for commit in tqdm(C):
                                query = str(Commit(commit, name))
                                statement.write(re.sub('-{2,}','\-\-',query)+';\n')
                            #statement.write('END TRANSACTION;\n')
                    elif count > 10000:
                        with open('insert.txt', 'w') as statement:
                            for commit in tqdm(C):
                                query = str(Commit(commit, name))
                                statement.write(re.sub('-{2,}','\-\-',query)+';\n')
                        with open('insert.txt', 'r') as statement:
                            query = statement.read()
                            write.executescript(query)
                            print ('Well done!')
                            del query
                    else:
                        for commit in tqdm(C):
                            query = str(Commit(commit, name))
                            write.execute(query)
                    query = str(Repo(R, count))
                    write.execute(query)
                    con.commit()     
                    del C, R
                k += 1
        except:
            con.rollback() 
            print ('Something is wrong! {}', name)
            time.sleep(5)
            with open ('except.txt','a') as f:
                f.write(name+'\n')

In [ ]:
datetime.datetime.strptime('2012-09-04 21:54:08', '%Y-%m-%d %H:%M:%S')

# 3 Get Users

In [ ]:
class User:
    
    def __init__(self, U):
        self.user = U.author.login
        self.date = U.created_at
        self.location = U.location
        self.private = U.owner_private_repos
        self.public = U.public_repos
        
    def __str__(self):
        return 'INSERT into U (user, date, location, private, public) \
        VALUES (\"{}\",\"{}\",\"{}\",{},{})'.format(
            str(self.user),
            str(self.date),
            str(self.location),
            str(self.private),
            str(self.public)
            )

In [ ]:
def get_users():
    name_cur.execute('SELECT DISTINCT user FROM R')
    name = name.fetchone()
    k = 1
    while name:
        U = g.get_user(name[0])
        Rate = g.get_rate_limit().core
        query = str(User(U))
        write.execute(query)
        con.commit()
        Rate = g.get_rate_limit().core
        if Rate.remaining < 5:
            time.sleep((Rate.reset - datetime.datetime.utcnow()).seconds+1)
        if k % 1000: 
            print ('{}\t{}'.format(k, str(datetime.datetime.now())[11:19]))
        k += 1 

# 1 Get repos

In [4]:
import sqlite3
con = sqlite3.connect('git.db')
cur = con.cursor()

import datetime

from tqdm import tqdm_notebook as tqdm

from github import Github

g = Github("dkbrz", "mandarinka24")

import time
from IPython.display import clear_output

def dates():
    for i in range(2012, 2019):
        for j in range(1, 13):
            w = '0'+str(j)
            w = str(i)+'-'+w[-2:]
            if w > '2012-10': 
                yield w

def download_repo_info():
    global g, con, cur
    for date in tqdm(list(dates())):
        try:
            C = g.search_repositories('stars:>200 created:{}'.format(date), sort='stars', order='desc')
            C._PaginatedList__requester.per_page = 100
            count = C.totalCount
            print (date, count)
            Rate = g.get_rate_limit()
            core = Rate.core
            search = Rate.search
            if count//100 + 3 >= search.remaining:
                'Sleepy'
                time.sleep((Rate.search.reset - datetime.datetime.utcnow()).seconds+1)
            cur.execute('BEGIN TRANSACTION;')
            for repo in C:
                name = repo.full_name
                query = 'INSERT into Repos (full, date) VALUES (\"{}\",\"{}\")'.format(str(name), str(date))
                cur.execute(query)
            con.commit()
        except:
            con.rollback()
            time.sleep(10)
            C = g.search_repositories('stars:>200 created:{}'.format(date), sort='stars', order='desc')
            C._PaginatedList__requester.per_page = 100
            count = C.totalCount
            print (date, count)
            Rate = g.get_rate_limit()
            core = Rate.core
            search = Rate.search
            if count//100 + 3 >= search.remaining:
                'Sleepy'
                time.sleep((Rate.search.reset - datetime.datetime.utcnow()).seconds+1)
            cur.execute('BEGIN TRANSACTION;')
            for repo in C:
                name = repo.full_name
                query = 'INSERT into Repos (full, date) VALUES (\"{}\",\"{}\")'.format(str(name), str(date))
                cur.execute(query)
            con.commit()
        clear_output()
        clear_output()

In [5]:
download_repo_info()